In [1]:
import csv, os
import cv2
import numpy as np
import matplotlib as pyplot

%matplotlib inline
### Read/parse the driving logfile
#print (os.getcwd())
lines = []
src_path = r'C:\Users\toshiba\Desktop\data'
log_file = src_path + r'\driving_log.csv'
img_dir = src_path + r'\IMG'
with open (log_file, 'r') as f:
    reader = csv.reader(f)
    flag = 0
    for line in reader:
        lines.append(line)

# Setup image and steerig angle training data
images = []
steer_angle = []
count = 0
angle_correction = 0.2
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('\\')[-1]
        current_path = img_dir + r'\\' + filename
        if (count == 3): print (current_path)
        image = cv2.imread(current_path)
        images.append(image)
        angle = float(line[3])
        if (i == 1):   angle = angle + 0.2
        elif (i == 2): angle = angle - 0.2
        steer_angle.append(angle)
        count = count + 1
    
X_train = np.array(images)
y_train = np.array(steer_angle)


### Data Augmentation to help driving steer Car clockwise direction
### Flip the images horizontally and change polarity of the steering angle

'''
aug_images, aug_steer_angle = [], []
for img, angle in zip (images, steer_angle):
    aug_images.append(img)
    aug_steer_angle.append(angle)
    aug_images.append(cv2.flip(img,1))
    aug_steer_angle.append(angle * -1.0)
    
X_train = np.array(aug_images)
y_train = np.array(aug_steer_angle)
'''


print ("# images, labels:", X_train.shape, len(y_train))
#img = np.random.randint(0, high=len(y_train))
#cv2.imshow("image %d" % (img), X_train[img])
#cv2.waitKey(0)
#cv2.destroyAllWindows()


C:\Users\toshiba\Desktop\data\IMG\\center_2017_11_07_22_38_12_012.jpg
# images, labels: (10668, 160, 320, 3) 10668


In [11]:
### Create a simple keras model
import timeit
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D


# record the execution time
start_time = timeit.default_timer()
print ("start time: %s seconds", start_time)
model = Sequential()
model.add (Cropping2D(cropping=((50,20),(0,0)), input_shape=(160,320,3)))
#model.add(Lambda(lambda x: (x/255.0) - 0.5, input_shape = (160, 320, 3)))
model.add(Lambda(lambda x: (x/255.0) - 0.5))
model.add(Convolution2D(6,5,5, activation = "relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile (loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle = 'True', nb_epoch = 5)

model.save('model.h5')
elapsed_time = timeit.default_timer() - start_time
print ("elapsed time: %s seconds", elapsed_time)

start time: %s seconds 519.728759644341
Train on 8534 samples, validate on 2134 samples
Epoch 1/5
8534/8534 [==============================] - 465s - loss: 0.9293 - val_loss: 0.0649
Epoch 2/5
8534/8534 [==============================] - 465s - loss: 0.0384 - val_loss: 0.0656
Epoch 3/5
6880/8534 [=======================>......] - ETA: 78s - loss: 0.0350

KeyboardInterrupt: 

In [99]:
drive.py model.h5


SyntaxError: invalid syntax (<ipython-input-99-04ca19bff3e5>, line 1)